## 基于MindSpore构建Min Pooling层

在本实验中，我们将实现最小池化，包括最大池化API的调用，实现最小池化API。

### 1、实验目的
- 掌握最小池化的原理。
- 掌握MindSpore中最大池化API的调用。
- 基于MindSpore实现二维最小池化。

### 2、最小池化原理介绍
在池化过程中，MinPooling采样输出区域的最小值，MaxPooling输出区域的最大值，MaxPooling的原理如下：

输入数据的维度为$(N_{in},C_{in},H_{in},W_{in})$，MaxPooling在$(H_{in},W_{in})$维度输出区域最大值。给定为$kernel\_size$为 $(kH,kW)$和 $stride$，运算如下所示：

$$MaxPool(N_i,C_j,h,w)=\underset{m=0,...kH-1}{max} \underset{n=0,...kW-1}{max} input(N_i,C_j,stride[0]\times h+m,stride[1]\times w+n )$$
其中$kH$和$kW$是池化核尺寸大小，$stride[0]$和$stride[1]$为池化操作的移动步长，$h$和$w$为最大池化输出的索引。

与MaxPooling相反，由于MinPooling使用场景极少，因此MindSpore中并未提供最小池化的API，但是提供了MaxPool，MindSpore框架中的MaxPool类对输入的多维数据进行二维的最大池化运算，因此可通过对输入数据的负值求最大池化，然后再求负值即可：$MinPool = -MaxPool(-x)$，其中x为输入的数据。

因此定义类class MinPool2d(_PoolNd)调用最大池化实现二维最小池化，定义类_PoolNd用于检查输入是否规范。

### 3 实验环境
在动手进行实践之前，需要注意以下几点：
* 确保实验环境正确安装，包括安装MindSpore。安装过程：首先登录[MindSpore官网安装页面](https://www.mindspore.cn/install)，根据安装指南下载安装包及查询相关文档。同时，官网环境安装也可以按下表说明找到对应环境搭建文档链接，根据环境搭建手册配置对应的实验环境。
* 推荐使用交互式的计算环境Jupyter Notebook，其交互性强，易于可视化，适合频繁修改的数据分析实验环境。
* 实验也可以在华为云一站式的AI开发平台ModelArts上完成。
* 推荐实验环境：MindSpore版本=2.0；Python环境=3.7


|  硬件平台 |  操作系统  | 软件环境 | 开发环境 | 环境搭建链接 |
| :-----:| :----: | :----: |:----:   |:----:   |
| CPU | Windows-x64 | MindSpore2.0 Python3.7.5 | JupyterNotebook |[MindSpore环境搭建实验手册第二章2.1节和第三章3.1节](./MindSpore环境搭建实验手册.docx)|
| GPU CUDA 10.1|Linux-x86_64| MindSpore2.0 Python3.7.5 | JupyterNotebook |[MindSpore环境搭建实验手册第二章2.2节和第三章3.1节](./MindSpore环境搭建实验手册.docx)|
| Ascend 910  | Linux-x86_64| MindSpore2.0 Python3.7.5 | JupyterNotebook |[MindSpore环境搭建实验手册第四章](./MindSpore环境搭建实验手册.docx)|


### 4、数据处理
#### 4.1 数据准备
简单起见，我们直接构建一个1x2x4x4的张量，调用np.random.randint函数生成随机数，值的范围在0到10。


Tensor是可放在gpu上加速的张量；dtype用于创建MindSpore中的数据类型对象；numpy是Python中科学计算的核心库,它提供了一个高性能的多维数组对象，以及用于处理这些数组的工具。

In [4]:
from mindspore import Tensor  
from mindspore import dtype as mstype    
import numpy as np                          

In [5]:
np.random.seed(1)
# 1x2x4x4的数据
x = Tensor(np.random.randint(0, 10, [1, 2, 4, 4]), mstype.float32)       

#### 4.2 数据加载

构建的数据如下所示：

In [6]:
print(x)

[[[[5. 8. 9. 5.]
   [0. 0. 1. 7.]
   [6. 9. 2. 4.]
   [5. 2. 4. 2.]]

  [[4. 7. 7. 9.]
   [1. 7. 0. 6.]
   [9. 9. 7. 6.]
   [9. 1. 0. 1.]]]]


### 5、模型构建
模型构建分为类_PoolND与MinPool2d。其中_PoolND用于检查输入pad_mode、data_format、kernel_size、stride是否符合规范；MinPool2d用于实现最小池化功能，MinPool2d将继承_PoolND实现输入检查，调用MindSpore框架中的最大池化API实现最小池化功能。
- 定义_PoolND类检查输入是否规范

mindspore.nn用于构建神经网络中的预定义构建块或计算单元；ops提供计算函数和构建神经网络的一些方法；Cell是MindSpore中神经网络的基本构成单元，模型或神经网络层应当继承该基类。

In [7]:
import mindspore.nn as nn
from mindspore import ops
import mindspore
from mindspore.nn import Cell

类_PoolNd用于检查输入是否规范，会检查pad_mode是否为VALID或SAME，数据格式是否为NCHW或NHWC，kernel_size和stride是否为正值。其中数据格式NHWC仅支持GPU，kernel_size和stride可以是一个值也可以是两个值。

In [8]:
class _PoolNd(Cell):

    def __init__(self, kernel_size, stride, pad_mode, data_format="NCHW"):
        super(_PoolNd, self).__init__()         
        # 检查pad_mode是否为VALID或SAME
        if pad_mode != 'VALID' and pad_mode != 'SAME':
            raise ValueError('The pad_mode must be VALID or SAME')
        self.pad_mode = pad_mode
        # 检查data_format是否为NCHW或NHWC
        if data_format !='NCHW' and data_format != 'NHWC':
            raise ValueError('The format must be NCHW or NHWC')          
        self.format = data_format
        # NHWC数据格式仅支持GPU
        if mindspore.get_context("device_target") != "GPU" and self.format == "NHWC":
            raise ValueError(f"For '{self.cls_name}, the 'NHWC' format only support in GPU target, but got device "    
                             f"target {mindspore.get_context('device_target')}.")

        # 检查是否为int或tuple，且必须为正数
        def _check_int_or_tuple(arg_name, arg_value):                                                                  
            error_msg = f"For '{self.cls_name}', the '{arg_name}' must be an positive int number or " \
                        f"a tuple of two positive int numbers, but got {arg_value}"
            if isinstance(arg_value, int):
                if arg_value <= 0:
                    raise ValueError(error_msg)
            elif isinstance(arg_value, tuple):
                if len(arg_value) == 2:
                    for item in arg_value:
                        if isinstance(item, int) and item > 0:
                            continue
                        raise ValueError(error_msg)
                else:
                    raise ValueError(error_msg)
            else:
                raise ValueError(error_msg)
            return arg_value
        
        # kernel_size是一个正数或两个正数的元组
        self.kernel_size = _check_int_or_tuple('kernel_size', kernel_size)  
        # stride是一个正数或两个正数的元组
        self.stride = _check_int_or_tuple('stride', stride)                           

    def construct(self, *inputs):
        pass

    def extend_repr(self):
        return 'kernel_size={kernel_size}, stride={stride}, pad_mode={pad_mode}'.format(**self.__dict__)

- 定义MinPool2d类实现二维最小池化

继承类_PoolNd检查输入，调用MindSpore的最大池化API处理输入数据的负值，然后再求负值就是最小池化的输出。

In [9]:
class MinPool2d(_PoolNd):
    def __init__(self, kernel_size=1, stride=1, pad_mode="VALID", data_format="NCHW"):
        # 检查输入是否规范
        super(MinPool2d, self).__init__(kernel_size, stride, pad_mode, data_format) 
        # 最大池化
        self.max_pool = ops.MaxPool(kernel_size=self.kernel_size,                          
                                  strides=self.stride,
                                  pad_mode=self.pad_mode,
                                  data_format=self.format)



    def construct(self, x):
        # minpool=- maxpool(-data)
        out = -self.max_pool(-x)                                                         
        return out

### 6、模型测试
我们进行算子测试，使用数据x进行最小池化和最大池化，发现输出结果符合预期。

In [10]:
# 直接调用定义好的最小池化类
pool = MinPool2d(kernel_size=3, stride=1)                                                 
pool_max=nn.MaxPool2d(kernel_size=3, stride=1)
print(x)
# 最小池化
output = pool(x)                                                                          
print(output)
# 最大池化
print(pool_max(x))                                                                        

[[[[5. 8. 9. 5.]
   [0. 0. 1. 7.]
   [6. 9. 2. 4.]
   [5. 2. 4. 2.]]

  [[4. 7. 7. 9.]
   [1. 7. 0. 6.]
   [9. 9. 7. 6.]
   [9. 1. 0. 1.]]]]
[[[[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]]]
[[[[9. 9.]
   [9. 9.]]

  [[9. 9.]
   [9. 9.]]]]
